# Evaluating Amazon Transcribe
In this notebook we evaluate Amazon Transcribe. The notebook assumes that test data is uploaded to an S3 bucket and a endpoint has been created.

In [1]:
import sagemaker
import boto3

sess = sagemaker.Session()
sagemaker_session_bucket = sess.default_bucket()

Getting the list of audio files in the S3 bucket which we will use for testing

In [2]:
s3 = boto3.client('s3')
file_list = []

response = s3.list_objects_v2(
    Bucket=sagemaker_session_bucket,
    Prefix='whisper/data/test/he/')

for content in response.get('Contents', []):
    if "wav" in content['Key']:
        file_list.append(f"s3://{sagemaker_session_bucket}/{content['Key']}")

In [3]:
len(file_list)

100

In [4]:
file_list[:5]

['s3://sagemaker-us-east-1-905847418383/whisper/data/test/he/test-he-000.wav',
 's3://sagemaker-us-east-1-905847418383/whisper/data/test/he/test-he-001.wav',
 's3://sagemaker-us-east-1-905847418383/whisper/data/test/he/test-he-002.wav',
 's3://sagemaker-us-east-1-905847418383/whisper/data/test/he/test-he-003.wav',
 's3://sagemaker-us-east-1-905847418383/whisper/data/test/he/test-he-004.wav']

This code will kick off a Amazon Transcribe job for each audio file and store the results in a text file in our S3 bucket

In [5]:
from sagemaker.utils import name_from_base

job_name_base = name_from_base("transcribe-hebrew")
transcribe = boto3.client('transcribe')

for i, f in enumerate(file_list):
    job_name = f"{job_name_base}-{i}"

    # Start the transcription job
    transcribe.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={'MediaFileUri': f},
        OutputBucketName=sagemaker_session_bucket,
        # transcription will be stored in txt file with same name as the wav file
        OutputKey=f'whisper/data/test/he/transcribe/{f.split("/")[-1].split(".")[0]}.txt',
        MediaFormat='wav',
        LanguageCode='he-IL'
    )

    print(f"Started transcription job: {job_name}")

Started transcription job: transcribe-hebrew-2022-12-09-10-29-42-753-0
Started transcription job: transcribe-hebrew-2022-12-09-10-29-42-753-1
Started transcription job: transcribe-hebrew-2022-12-09-10-29-42-753-2
Started transcription job: transcribe-hebrew-2022-12-09-10-29-42-753-3
Started transcription job: transcribe-hebrew-2022-12-09-10-29-42-753-4
Started transcription job: transcribe-hebrew-2022-12-09-10-29-42-753-5
Started transcription job: transcribe-hebrew-2022-12-09-10-29-42-753-6
Started transcription job: transcribe-hebrew-2022-12-09-10-29-42-753-7
Started transcription job: transcribe-hebrew-2022-12-09-10-29-42-753-8
Started transcription job: transcribe-hebrew-2022-12-09-10-29-42-753-9
Started transcription job: transcribe-hebrew-2022-12-09-10-29-42-753-10
Started transcription job: transcribe-hebrew-2022-12-09-10-29-42-753-11
Started transcription job: transcribe-hebrew-2022-12-09-10-29-42-753-12
Started transcription job: transcribe-hebrew-2022-12-09-10-29-42-753-13
St

Once the jobs are finished (we can check the status in the Transcribe console) we can collect the output.

In [7]:
import boto3
import json

client = boto3.client('s3')
preds_transcribe = []

response = client.list_objects_v2(
    Bucket=sagemaker_session_bucket,
    Prefix='whisper/data/test/he/transcribe/')

for o in response.get('Contents'):
    if "txt" in o.get('Key'):
        data = client.get_object(Bucket=sagemaker_session_bucket, Key=o.get('Key'))
        contents = data['Body'].read()
        content = contents.decode("utf-8")
        d = json.loads(content)
        preds_transcribe.append(d['results']['transcripts'][0]['transcript'])

In [8]:
preds_transcribe[:5]

['אזורים גדולים צפונית משם הם בעלי אוכלוסיית גדילה יחסית וחלק הם אזורי פרק כמעט לא מיושבים.',
 "כתוצאה מכך השחקנים מעשנים ג'וינטים של קנאביס על הבמה והתיאטרון עצמו מעודד את הקהל להצטרף.",
 'הלכות על הידיים שלך טבעיים השכבות החיצוניות, מה שיגרום לתחושה מוזרות ויצוא. מעין קליפה.',
 'בתקיפות בתחילת גברים. ואל תפחדו לעמוד לשלחם הבדלים תרבותיים או לא. זה לא הופך את זה לבסדר.',
 "דוקטור 'מעלה' בלאס בו מניין בן נמצא בבל הוא אש או היום פרווה נמצא כ קילומטר צפונית לסינסנטי כשהוא שוכב על האדמה בצד הדרך בחולצת טריקו ותחתונים, ככל הנראה תחת השפעה כבדה של תרופות."]

Downloading the csv file that contains the ground truth

In [9]:
s3.download_file(sagemaker_session_bucket, "whisper/data/test/he/test-he.tsv", 'test-he.tsv')

In [11]:
import pandas as pd
df = pd.read_csv('test-he.tsv', sep='\t')

In [12]:
df["transcription_transcribe"] = preds_transcribe

## Overall WER score

In [15]:
predictions = list(df["transcription_transcribe"])
references = list(df["raw_transcription"])

In [16]:
import evaluate
wer = evaluate.load("wer")

In [17]:
wer_score = wer.compute(predictions=predictions, references=references)
wer_score

0.4025157232704403

## Individual WER Score

In [18]:
def compute_wer(row, col1, col2):
    return wer.compute(predictions=[row[col1]], references=[row[col2]])

In [20]:
df["WER score - Transcribe"] = df.apply(compute_wer, args=('transcription_transcribe', 'raw_transcription'), axis=1)

In [22]:
df.sort_values("WER score - Transcribe", ascending=False).head(10)

,id,raw_transcription,transcription,new_filename,transcription_transcribe,WER score - Transcribe
36,1885,בין השעות 22:00-23:00 לפי אזור זמן ההרים (MDT)...,בין השעות 22:00-23:00 לפי אזור זמן ההרים mdt ה...,test-he-036.wav,פשוט לחץ על זה לפני סוף השבוע.,1.000000
18,1850,"זה יאפשר לו להיות תואם לאחור עם 802.11a, 802.1...",זה יאפשר לו להיות תואם לאחור עם 802.11a 802.11...,test-he-018.wav,בתנאי שלתחנת הבסיס יש מכשירי קשר כפולים. מי זה...,1.000000
65,1668,"ייתכן גם שכדאי לרכוש Wild Card, המאפשר כניסה ל...",ייתכן גם שכדאי לרכוש wild card המאפשר כניסה למ...,test-he-065.wav,"המאפשר כניסה למבחר פארקים. בדומה, אפריקה. לא ל...",0.888889
80,1870,הסצנות מוצגות על גבי הפירמידות והפירמידות השונ...,הסצנות מוצגות על גבי הפירמידות והפירמידות השונ...,test-he-080.wav,מצאנו את הצוות לגבי פעמינו והתלמידות השונות מרור.,0.875000
55,1735,חוקרים מאוניברסיטת פרינסטון בארצות הברית ואוני...,חוקרים מאוניברסיטת פרינסטון בארצות הברית ואוני...,test-he-055.wav,חוקרים באוניברסיטת פרינסטון בארצות הברית באוני...,0.846154
33,1827,"עובי הקרום הוא כ-70 ק""מ בצד הקרוב וכ-100 ק""מ ב...",עובי הקרום הוא כ-70 ק מ בצד הקרוב וכ-100 ק מ ב...,test-he-033.wav,רובי היקום הוא כי שבעים קילומטר בצד הקרוב והיא...,0.818182
48,1814,יוני מימן הם פרוטונים שהאלקטרונים שלהם הופשטו ...,יוני מימן הם פרוטונים שהאלקטרונים שלהם הופשטו ...,test-he-048.wav,יונה ממנה בפרוטונים שהאלקטרונים שלהם הופשטו מה...,0.812500
73,1672,היזהרו שהבד לא יתחמם יותר מידי (מה שיכול לגרום...,היזהרו שהבד לא יתחמם יותר מידי מה שיכול לגרום ...,test-he-073.wav,היזהרו שהבד להתחבר יותר מדי.,0.785714
86,1954,"ההגנה הממושמעת, יכולות השליטה בכדור ועבודת הצו...",ההגנה הממושמעת יכולות השליטה בכדור ועבודת הצוו...,test-he-086.wav,ההגנה ממושמע את יכולות השליטה בכדור עבודת צוות...,0.777778
39,1918,יער סונדרבאנס הוכרז כאתר מורשת עולמי מטעם יונס...,יער סונדרבאנס הוכרז כאתר מורשת עולמי מטעם יונס...,test-he-039.wav,יופי. שום דבר זוכרת. ואתר מורשת עולמי מטעם יונ...,0.764706
